In [30]:
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/gibsonjackson/FODSIMG/main/TrainFinal.csv')
df=(df-df.min())/(df.max()-df.min())
fd = df
#df.head()
#df.info()
Res = df['quantitative response of LC50']
X1 = df['MLOGP']
# X1 = (X1-X1.mean())/X1.std()
X2 = df['RDCHI']
# X2 = (X2-X2.mean())/X2.std()
Y1 = Res.tolist()
X1 = X1.tolist()
X2 = X2.tolist()


dfv = pd.read_csv('https://raw.githubusercontent.com/gibsonjackson/FODSIMG/main/Validate.csv')
dfv=(dfv-dfv.min())/(dfv.max()-dfv.min())
fdv = df
#dfv.head()
#dfv.info()
Resv = dfv['quantitative response of LC50']
X1v = dfv['MLOGP']
X2v = dfv['RDCHI']
Y1v = Resv.tolist()
X1v = X1v.tolist()
X2v = X2v.tolist()


dft = pd.read_csv('https://raw.githubusercontent.com/gibsonjackson/FODSIMG/main/Test.csv')
dft=(dft-dft.min())/(dft.max()-dft.min())
fdt = df
#dft.head()
#dft.info()
Rest = dft['quantitative response of LC50']
X1t = dft['MLOGP']
X2t = dft['RDCHI']
Y1t = Rest.tolist()
X1t = X1t.tolist()
X2t = X2t.tolist()

*Creating polynomial stochastic regression model with regularisation*

In [31]:
def hypothesis(X1,X2, theta,degree):
  Pred_Y = [0.0]*len(X1)
  for y in range(len(Pred_Y)):
    for i in range(degree+1):
      for j in range(degree+1):
        if (i+j>degree or i==j):
          break
        Pred_Y[y] = Pred_Y[y] + (theta[i][j]*pow(X1[y],i)*pow(X2[y],j))
  return Pred_Y

In [32]:
def cost(X1,X2,Y,theta,degree):
    y1 = hypothesis(X1,X2,theta,degree)
    return sum(2*(y1-np.array(Y))**2)/(len(X1))

In [33]:
def stochasticLinearRegression(X1,X2,Y,alpha,epoch,degree,reg,q):
  theta = [ [1.0]*(degree+1) for i in range(degree+1)]
  J = []
  k = 0
  size = len(X1)
  x1=X1
  x2=X2
  yu = Y
  while k<epoch:
    y1 = hypothesis(X1,X2,theta,degree) #Y_pred matrix from equation using theta1x1+theta2x2+theta3
    yy = y1
    for r in range(size):
      for i in range(degree+1):
        for j in range(degree+1):
          if (i+j>degree):
            break
          
          theta[i][j]=theta[i][j]-alpha*((yy[r]-yu[r])*(x1[r]**i)*(x2[r]**j)+reg*((i*theta[i][j]**(q-1))+(j*theta[i][j]**(q-1))))/size
          

    k+=1
    J.append(cost(X1,X2,Y,theta,degree))
    if (k>1 and J[len(J)-2]-J[len(J)-1]<1e-6):
        break
  return J,theta

In [34]:
from tabulate import tabulate
allTheta=[]
Jall = []
qval = [0.5,1,2,4]

In [35]:
def all_q(ref):
  Jq=[]
  TestError=[]
  print(ref)
  for i in qval:
    
    Jp,TP = stochasticLinearRegression(X1,X2,Y1,0.001,10000,1,ref,i)
    Jq.append(Jp[len(Jp)-1])
    TestError.append(cost(X1t,X2t,Y1t,TP,1))
  print(TestError)

In [ ]:
i = 0.05
j = 0
while j<=1:
  all_q(j)
  j+=i